# 微调 (Fine Tuning)
微调（Fine Tuning）是深度学习中的一种迁移学习技术，通过在预训练模型的基础上，针对特定任务进行进一步的训练，以提高模型在该任务上的表现。微调通常用于以下场景：

    *数据集较小：*当目标任务的数据集较小，直接从头开始训练模型容易导致过拟合。

    *计算资源有限：*从头开始训练一个深度神经网络需要大量的计算资源和时间，而微调可以节省这些资源。

## 微调的基本步骤
1. 选择预训练模型：

    - 选择一个在大型数据集（如ImageNet）上预训练好的模型，如ResNet、VGG、BERT等。

2. 冻结部分层：

    - 通常情况下，我们会冻结预训练模型的前几层，只对后面的几层进行微调。这是因为前几层通常学习到的是通用的特征（如边缘、纹理等），而后几层学习到的是更高级的特征（如物体部分、类别等）。

3. 替换输出层：

    - 根据目标任务的需求，替换预训练模型的输出层。例如，如果预训练模型是用于图像分类的，而目标任务是目标检测，则需要替换输出层以适应新的任务。

4. 微调训练：

    - 使用目标任务的数据集对模型进行进一步训练。通常会使用较小的学习率，以避免破坏预训练模型已经学习到的特征。

5. 评估和调整：

    - 在验证集上评估模型的性能，并根据需要调整超参数（如学习率、冻结层数等）。

![微调](https://zh.d2l.ai/_images/finetune.svg "Fine Tuning")

## 微调的优点
1. 提高模型性能：通过利用预训练模型的知识，微调可以显著提高模型在目标任务上的性能。

2. 减少训练时间：微调通常比从头开始训练模型更快，因为它只需要对部分层进行训练。

3. 适用于小数据集：微调特别适用于数据集较小的情况，因为它可以利用预训练模型的知识来弥补数据的不足。

## 微调的注意事项
1. 学习率选择：微调时通常使用较小的学习率，以避免破坏预训练模型的特征。

2. 冻结层数：冻结的层数需要根据具体任务和数据集进行调整，过多的冻结可能导致模型无法适应新任务，而过少的冻结可能导致过拟合。

3. 数据增强：在微调过程中，使用数据增强技术可以进一步提高模型的泛化能力。



## 简单代码实现

In [12]:
%matplotlib inline
import os
import torch
import torchvision
from torch import nn
from d2l import torch as d2l

In [13]:
pretrained_net = torchvision.models.resnet18(pretrained=True)

In [14]:
pretrained_net.fc

Linear(in_features=512, out_features=1000, bias=True)

In [15]:
finetune_net = torchvision.models.resnet18(pretrained=True)
finetune_net.fc = nn.Linear(finetune_net.fc.in_features, 2)
nn.init.xavier_uniform_(finetune_net.fc.weight);

In [16]:
# 如果param_group=True，输出层中的模型参数将使用十倍的学习率
def train_fine_tuning(net, learning_rate, batch_size=128, num_epochs=5,
                      param_group=True):
    train_iter = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(
        os.path.join(data_dir, 'train'), transform=train_augs),
        batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(
        os.path.join(data_dir, 'test'), transform=test_augs),
        batch_size=batch_size)
    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
        trainer = torch.optim.SGD([{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'lr': learning_rate * 10}],
                                lr=learning_rate, weight_decay=0.001)
    else:
        trainer = torch.optim.SGD(net.parameters(), lr=learning_rate,
                                  weight_decay=0.001)
    d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,
                   devices)

In [ ]:
train_fine_tuning(finetune_net, 5e-5)